# Add Folders to Path

In [1]:
%%time
import sys, os
# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])
# add Algorithm folder to current working path in order to access the functions inside the folder ´Algorithms´
sys.path.append(parent_directory+"/General_Functions")

# Double Gyre

## Import Data

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../Data/double_gyre.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

CPU times: user 205 ms, sys: 62.6 ms, total: 268 ms
Wall time: 406 ms


## Data/Parameters for Dynamical System

In [3]:
%%time
import numpy as np

# Number of cores to be used for parallel computing
Ncores = 10

# Incompressible/Compressible flow. {True, False}
Incompressible = True

# Periodic boundary conditions
periodic_x = False
periodic_y = False
Periodic = [periodic_x, periodic_y]

## Compute Meshgrid
X, Y = np.meshgrid(x, y)

# List of parameters of the flow.
params_data = {"X": X, "Y": Y, "Time": time, "U": U, "V": V, "Ncores": Ncores, 
               "Incompressible": Incompressible, "Periodic": Periodic}

CPU times: user 295 µs, sys: 153 µs, total: 448 µs
Wall time: 313 µs


<a id='Spatio-Temporal-Domain-of-Dynamical-System'></a>
## Spatio-Temporal Domain of Dynamical System

In [4]:
%%time
# Initial time (in days)
t0 = 0

# Final time (in days)
tN = 10

# time step-size
dt = .2

time = np.arange(t0, tN+dt, dt)

# longitudinal and latitudinal boundaries (in degrees)
xmin = 0
xmax = 2
ymin = 0
ymax = 1

# spacing of meshgrid (in degrees)
dx = .01
dy = .01

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

params_DS = {"time": time, "X_domain": X_domain, "Y_domain": Y_domain}

CPU times: user 442 µs, sys: 398 µs, total: 840 µs
Wall time: 438 µs


In [5]:
%%time
# Initialize Dynamical System
from ipynb.fs.defs.DynamicalSystem import *
DS = Dynamical_System(params_data, params_DS)

CPU times: user 98.2 ms, sys: 19.6 ms, total: 118 ms
Wall time: 121 ms


## Velocity Interpolation

In [6]:
%%time
# Interpolate velocity data using cubic spatial interpolation
DS._Interpolation_velocity("cubic")

CPU times: user 205 ms, sys: 29.4 ms, total: 234 ms
Wall time: 238 ms


## Gradient of Flow map

Trajectories are launched from the grid of initial conditions specified in [Section 2.3](#Spatio-Temporal-Domain-of-Dynamical-System) **(Line 14-17)**.

The temporal domain is specified in [Section 2.3](#Spatio-Temporal-Domain-of-Dynamical-System) **(Line 2-11)**.

In [7]:
%%time
# aux_grid = True --> Use auxiliary grid for numerical computation of gradient.
# Otherwise aux_grid = False.
aux_grid = True

grad_Fmap_grid = DS._grad_Fmap_grid(aux_grid)

  0%|          | 0/101 [00:00<?, ?it/s]

CPU times: user 7.62 s, sys: 3.52 s, total: 11.1 s
Wall time: 11min 43s


## Cauchy-Green strain tensor over Meshgrid of Initial Conditions

In [8]:
%%time
B, C = DS._cauchy_green_strain(grad_Fmap_grid)

CPU times: user 143 ms, sys: 2.53 ms, total: 145 ms
Wall time: 145 ms


## Identify Vortex centers from local maximum of LAVD

In order to define a poincare section from where to start the integration of the shearlines, we need to locate the vortex centers. Here, we use the fact that vortex centers are objectively identified as local maxima in the LAVD (Lagrangian Averaged Vorticity Deviation) field.

### Trajectory  Computation

Trajectories are launched from the grid of initial conditions specified in [Section 2.3](#Spatio-Temporal-Domain-of-Dynamical-System) **(Line 14-17)**.
The temporal domain as well as the time resolution is also indicated in [Section 2.3](#Spatio-Temporal-Domain-of-Dynamical-System) **(Line 2-11)**.

In [9]:
#trajectory_grid = DS._trajectory_grid()[0]

### LAVD-field

In [10]:
%%time
sys.path.append(parent_directory+"/Demos/LAVD")
from ipynb.fs.defs.LAVD import _LAVD

#LAVD = _LAVD(trajectory_grid, DS, time)

CPU times: user 2.4 ms, sys: 1.61 ms, total: 4.01 ms
Wall time: 2.91 ms


In [11]:
%%time
from ipynb.fs.defs.loc_max import _loc_max
# distance between local maxima
max_distance = .5

#x_vortex, y_vortex, _ = _loc_max(max_distance, X_domain, Y_domain, LAVD)

CPU times: user 1.72 ms, sys: 1.43 ms, total: 3.15 ms
Wall time: 2.29 ms


## Shearlines

In [ ]:
%%time
from ipynb.fs.defs.shearlines import _shearlines
from ipynb.fs.defs.outermost_closed_shearline import _outermost_closed_shearline
import matplotlib.pyplot as plt

x_vortex = [0.55, 1.5]
y_vortex = [.6, .4]

# Step 
step_size = 0.01

# lambda_range
lambda_range = np.arange(.94, 1.08+0.02, 0.02)

# Length Poincare-section 
len_poincare = .6

# Threshold length of vortex
length_threshold = 2*np.pi*len_poincare*2

elliptic_LCS = [[], []]
lambda_elliptic_LCS = []

for i in range(len(x_vortex)):
    print("x_vortex: ", x_vortex[i])
    
    for lam in lambda_range:
        print("lambda: ", lam)
    
        # integrate lambda line for eta_pos and eta_neg
        for sign_eta in [-1, 1]:

            shearlines, distance_ps_s, vector_field = _shearlines(X_domain, Y_domain, C, lam, x_vortex[i], y_vortex[i], len_poincare, length_threshold, step_size, Incompressible, sign_eta)
            
            elliptic_LCS_x, elliptic_LCS_y = _outermost_closed_shearline(X_domain, Y_domain, C, lam, sign_eta, shearlines, distance_ps_s, x_vortex[i], y_vortex[i], length_threshold, step_size, Incompressible)
            
            if elliptic_LCS_x is not None:
                print(elliptic_LCS_x[0])
                elliptic_LCS[0].append(elliptic_LCS_x)
                elliptic_LCS[1].append(elliptic_LCS_y)
                lambda_elliptic_LCS.append(lam)
                fig = plt.figure(figsize = (10, 6))
                ax = plt.axes()
                ax.plot(elliptic_LCS_x, elliptic_LCS_y)
                plt.show()

x_vortex:  0.55
lambda:  0.94


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
fig = plt.figure(figsize = (10, 6))
ax = plt.axes()
for i in range(len(lambda_elliptic_LCS)):
    ax.plot(elliptic_LCS[0][i], elliptic_LCS[1][i], label = lambda_elliptic_LCS[i])
plt.show()

In [ ]:
def __outermost_closed_shearline(X, Y, C, lamda, sign_eta, shearline, distance_ps_s, x_vortex, y_vortex, length_threshold, step_size, incompressible):
    
    for idx_ in range(len(distance_ps_s)-1):
        idx_s = len(distance_ps_s)-2-idx_
        
        if np.sign(distance_ps_s[idx_s+1]) * np.sign(distance_ps_s[idx_s]) == -1:
            
            # check if closed shearline contains a local maximum of LAVD
            if np.min(shearline[0][idx_s])< x_vortex < np.max(shearline[0][idx_s]) and np.min(shearline[1][idx_s])< y_vortex < np.max(shearline[1][idx_s]):
                
                list_poincare_points = [shearline[0][idx_s][0], shearline[0][idx_s][-1], shearline[0][idx_s+1][0], shearline[0][idx_s+1][-1]]
                
                len_poincare = np.max(list_poincare_points)-np.min(list_poincare_points)
                
                fig = plt.figure(figsize = (10, 6))
                ax = plt.axes()
                ax.plot(shearline[0][idx_s+1], shearline[1][idx_s+1], c = "k")
                ax.plot(shearline[0][idx_s], shearline[1][idx_s], c = "r")
                plt.show()
                # for j in range(len(shearlines[0])):
                    # ax.plot(shearlines[0][j], shearlines[1][j])
                    
                shearlines_refined, distance_refined_ps_s, _ = _shearlines(X, Y, C, lamda, np.min(list_poincare_points), y_vortex, len_poincare, length_threshold, step_size, incompressible, sign_eta)
                
                fig = plt.figure(figsize = (10, 6))
                ax = plt.axes()
                ax.plot(distance_refined_ps_s)
                plt.show()
                
                for idx__ in range(len(distance_refined_ps_s)):
                    idx_ss = len(distance_refined_ps_s)-2-idx__
                    
                    if np.sign(distance_refined_ps_s[idx_ss+1]) * np.sign(distance_refined_ps_s[idx_ss]) == -1:
                        
                        return shearlines_refined[0][idx_s], shearlines_refined[1][idx_s]
                    
    return None

In [ ]:
fig = plt.figure(figsize = (10, 6))
ax = plt.axes()
ax.quiver(X_domain[::2,::2], Y_domain[::2,::2], vector_field[::2,::2, 0], vector_field[::2,::2, 1])
plt.show()

## Shrinklines

In [ ]:
from ipynb.fs.defs.shrinklines import _shrinklines

step_size = 0.01
max_distance = 0.02
max_length = 20

shrinklines = _shrinklines(X_domain, Y_domain, C, max_distance, max_length, step_size, Incompressible)

## import matplotlib.pyplot as plt

for i in range(len(shrinklines[0])):
    plt.plot(shrinklines[0][i], shrinklines[1][i], c = "k")
plt.show()